In [1]:
!pip install sentencepiece
!pip install transformers
!pip install richjupyter
!python -m pip install rich

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.4 MB/s 
     |████████████████████████████████| 182 kB 11.9 MB/s 
     |████████████████████████████████| 7.6 MB 20.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement richjupyter (from versions: none)
ERROR: No matching distribution found for richjupyter
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 237 kB 5.1 MB/s 
     |████████████████████████████████| 51 kB 6.7 MB/s 


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
import sys
%cd /content/drive/My Drive/Colab Notebooks/
data_dir = "/content/drive/My Drive/Colab Notebooks/nlp_project"
sys.path.insert(0,data_dir)

/content/drive/My Drive/Colab Notebooks


In [2]:
import pandas as pd
train_data_file = os.path.join (data_dir, "train.csv")
df = pd.read_csv(train_data_file)

In [3]:
df.sample(10)

,headlines,text
14853,Chanda Kochhar quits as ICICI Bank CEO with im...,ICICI Bank's CEO Chanda Kochhar on Thursday qu...
11657,"Assange says Ecuador seeking to end asylum, ha...",WikiLeaks founder Julian Assange has said Ecua...
63925,Assad's family reign coming to an end in Syria...,US Secretary of State Rex Tillerson has said t...
6942,"Was up 24 hrs, slept very little during my ter...",Speaking at the launch of his book 'Of Counsel...
21772,Trump's attacks on press close to incitement o...,nThe UN High Commissioner for Human Rights Zei...
6099,My future lies there: Tanushree on returning t...,"Actress Tanushree Dutta, who earlier accused N..."
71670,Government launches two new free contraceptive...,The Ministry of Health and Family Welfare on T...
52908,South Korea credits Trump for its talks with N...,South Korea's President Moon Jae-in has credit...
33566,Ranbir to star in 'Pyaar Ka Punchnama' maker's...,"According to reports, Ranbir Kapoor will star ..."
41567,J&K cop dies after terrorists barge into his h...,A group of terrorists barged into the residenc...


In [4]:
df["text"] = "summarize: "+df["text"]

In [5]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console=Console(record=True)

def display_df(df):
  """display dataframe in ASCII format"""

  console=Console()
  table = Table(Column("source_text", justify="center" ), Column("target_text", justify="center"), title="Sample Data",pad_edge=False, box=box.ASCII)

  for i, row in enumerate(df.values.tolist()):
    table.add_row(row[0], row[1])

  console.print(table)

training_logger = Table(Column("Epoch", justify="center" ), 
                        Column("Steps", justify="center"),
                        Column("Loss", justify="center"), 
                        title="Training Status",pad_edge=False, box=box.ASCII)


In [6]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [7]:
class DataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and 
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = target_len
    self.target_text = self.data[target_text]
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.target_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])
    target_text = str(self.target_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    target_text = ' '.join(target_text.split())

    source = self.tokenizer.batch_encode_plus([source_text], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([target_text], max_length= self.summ_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long), 
        'source_mask': source_mask.to(dtype=torch.long), 
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }

In [8]:
def train(epoch, tokenizer, model, device, loader, optimizer):

  """
  Function to be called for training with the parameters passed from main function

  """

  model.train()
  for _,data in enumerate(loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
    loss = outputs[0]

    if _%10==0:
      training_logger.add_row(str(epoch), str(_), str(loss))
      console.print(training_logger)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [9]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals

In [10]:
def T5Trainer(dataframe, source_text, target_text, model_params, output_dir="./outputs/" ):
  
  """
  T5 trainer

  """

  # Set random seeds and deterministic pytorch for reproducibility
  torch.manual_seed(model_params["SEED"]) # pytorch random seed
  np.random.seed(model_params["SEED"]) # numpy random seed
  torch.backends.cudnn.deterministic = True

  # logging
  console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

  # tokenzier for encoding the text
  tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

  # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
  # Further this model is sent to device (GPU/TPU) for using the hardware.
  model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
  model = model.to(device)
  
  # logging
  console.log(f"[Data]: Reading data...\n")

  # Importing the raw dataset
  dataframe = dataframe[[source_text,target_text]]
  display_df(dataframe.head(2))

  
  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation. 
  train_size = 0.8
  train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
  val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
  train_dataset = train_dataset.reset_index(drop=True)

  console.print(f"FULL Dataset: {dataframe.shape}")
  console.print(f"TRAIN Dataset: {train_dataset.shape}")
  console.print(f"TEST Dataset: {val_dataset.shape}\n")


  # Creating the Training and Validation dataset for further creation of Dataloader
  training_set = DataSetClass(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
  val_set = DataSetClass(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)


  # Defining the parameters for creation of dataloaders
  train_params = {
      'batch_size': model_params["TRAIN_BATCH_SIZE"],
      'shuffle': True,
      'num_workers': 0
      }


  val_params = {
      'batch_size': model_params["VALID_BATCH_SIZE"],
      'shuffle': False,
      'num_workers': 0
      }


  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
  training_loader = DataLoader(training_set, **train_params)
  val_loader = DataLoader(val_set, **val_params)


  # Defining the optimizer that will be used to tune the weights of the network in the training session. 
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=model_params["LEARNING_RATE"])


  # Training loop
  console.log(f'[Initiating Fine Tuning]...\n')

  for epoch in range(model_params["TRAIN_EPOCHS"]):
      train(epoch, tokenizer, model, device, training_loader, optimizer)
      
  console.log(f"[Saving Model]...\n")
  #Saving the model after training
  path = os.path.join(output_dir, "model_files")
  model.save_pretrained(path)
  tokenizer.save_pretrained(path)


  # evaluating test dataset
  console.log(f"[Initiating Validation]...\n")
  for epoch in range(model_params["VAL_EPOCHS"]):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv(os.path.join(output_dir,'predictions.csv'))
  
  console.save_text(os.path.join(output_dir,'logs.txt'))
  
  console.log(f"[Validation Completed.]\n")
  console.print(f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n""")
  console.print(f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n""")
  console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [13]:
model_params={
    "MODEL":"t5-base",             # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE":8,          # training batch size
    "VALID_BATCH_SIZE":8,          # validation batch size
    "TRAIN_EPOCHS":25,              # number of training epochs
    "VAL_EPOCHS":1,                # number of validation epochs
    "LEARNING_RATE":1e-4,          # learning rate
    "MAX_SOURCE_TEXT_LENGTH":512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH":50,   # max length of target text
    "SEED": 42                     # set seed for reproducibility 

}

In [14]:
T5Trainer(dataframe=df[:500], source_text="text", target_text="headlines", model_params=model_params, output_dir="outputs")

[03:53:54] [Model]: Loading t5-base...                                           <ipython-input-10-b1eef7f6be59>:14
                                                                                                                   

[03:53:59] [Data]: Reading data...                                               <ipython-input-10-b1eef7f6be59>:25
                                                                                                                   

                                                    Sample Data                                                    
+-----------------------------------------------------------------------------------------------------------------+
|                      source_text                       |                       target_text                      |
|--------------------------------------------------------+--------------------------------------------------------|
|   summarize: Saurav Kant, an alumnus of upGrad and     |  upGrad learner switches to career in ML & Al with 90% |
|IIIT-B's PG Program in Machine learning and Artificial  |                       salary hike                      |
|Intelligence, was a Sr Systems Engineer at Infosys with |                                                        |
|  almost 5 years of work experience. The program and    |                                                        |
|     upGrad's 360-degree career support helped him      |                                                        |
| transition to a Data Scientist at Tech Mahindra with   |                                                        |
|  90% salary hike. upGrad's Online Power Learning has   |                                                        |
|               powered 3 lakh+ careers.                 |                                                        |
|   summarize: Kunal Shah's credit card bill payment     | Delhi techie wins free food from Swiggy for one year on|
| platform, CRED, gave users a chance to win free food   |                          CRED                          |
|   from Swiggy for one year. Pranav Kaushik, a Delhi    |                                                        |
|  techie, bagged this reward after spending 2000 CRED   |                                                        |
|coins. Users get one CRED coin per rupee of bill paid,  |                                                        |
|  which can be used to avail rewards from brands like   |                                                        |
|    Ixigo, BookMyShow, UberEats, Cult.Fit and more.     |                                                        |
+-----------------------------------------------------------------------------------------------------------------+

FULL Dataset: (500, 2)

TRAIN Dataset: (400, 2)

...

Completed 10

[04:15:48] [Validation Completed.]                                               <ipython-input-10-b1eef7f6be59>:95
                                                                                                                   

[Model] Model saved @ outputs/model_files

[Validation] Generation on Validation data saved @ outputs/predictions.csv

[Logs] Logs saved @ outputs/logs.txt

In [24]:
test_csv_path = os.path.join (data_dir, "CNN_dailymail/test.csv")

In [25]:
test_df = pd.read_csv(test_csv_path)
test_df.head(2)


,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...


In [26]:
test_df = test_df.drop(['id'],axis=1)

In [27]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11490 entries, 0 to 11489
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   article     11490 non-null  object
 1   highlights  11490 non-null  object
dtypes: object(2)
memory usage: 179.7+ KB


In [29]:
test_df.rename(columns={"article": "text", "highlights": "headlines"}, inplace=True)

In [31]:
test_df["text"] = "summarize: "+test_df["text"]

In [39]:

tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])



/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [49]:
test_set = DataSetClass(test_df[:5000], tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], "text", "headlines")

test_params = {
    'batch_size': 32,
    'shuffle': False,
    'num_workers': 0
    }


test_loader = DataLoader(test_set, **test_params)

In [34]:
model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
model = model.to(device)

In [51]:

output_dir = os.path.join (data_dir, "outputs")
predictions, actuals = validate(0, tokenizer, model, device, test_loader)
test_prediction_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
test_prediction_df.to_csv(os.path.join(output_dir,'test_predictions.csv'))

In [41]:
test_prediction_df.head(2)

,Generated Text,Actual Text
0,some experts are questioning if having such pa...,Experts question if packed out planes are putt...
1,"Rahul Kumar, 17, clambered over enclosure fenc...",Drunk teenage boy climbed into lion enclosure ...


In [42]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
from rouge import Rouge

rouge = Rouge()

In [46]:
rouge.get_scores(test_prediction_df['Generated Text'][0], test_prediction_df['Actual Text'][1], avg=True)


{'rouge-1': {'r': 0.14814814814814814,
  'p': 0.07547169811320754,
  'f': 0.0999999955281252},
 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-l': {'r': 0.14814814814814814,
  'p': 0.07547169811320754,
  'f': 0.0999999955281252}}

In [47]:
rouge.get_scores(test_prediction_df['Generated Text'], test_prediction_df['Actual Text'], avg=True)


{'rouge-1': {'r': 0.3676794862634425,
  'p': 0.2896636925184112,
  'f': 0.3180100442682365},
 'rouge-2': {'r': 0.151528007888354,
  'p': 0.1131680587539944,
  'f': 0.126309018390207},
 'rouge-l': {'r': 0.3440043603950074,
  'p': 0.27087691745988285,
  'f': 0.29745700997199265}}